In [1]:
import dpeeg
import numpy as np

dpeeg.set_seed()
print(dpeeg.__version__)

/home/tao/miniconda3/envs/bci/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/tao/miniconda3/envs/bci/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


0.3.6


In [ ]:
import numpy as np
from dpeeg.data import transforms

data = {
    1: {'train': [np.random.rand(64, 3, 10), np.random.randint(2, size=64)],
        'test':  [np.random.rand(16, 3, 10), np.random.randint(2, size=16)]},
    2: {'train': [np.random.rand(64, 3, 10), np.random.randint(2, size=64)],
        'test':  [np.random.rand(16, 3, 10), np.random.randint(2, size=16)]},
    3: {'train': [np.random.rand(64, 3, 10), np.random.randint(2, size=64)],
        'test':  [np.random.rand(16, 3, 10), np.random.randint(2, size=16)]},
}

for sub, mode, sub_data in transforms.yield_data(data, 'train'):
    print(sub, mode)

1 train
2 train
3 train


In [2]:
import numpy as np
from dpeeg.data.functions import segmentation_and_reconstruction

data = np.random.rand(5, 3, 8)
label = np.random.randint(0, 2, 5)
print(data, label)
print('-'*50)
aug_data, aug_label = segmentation_and_reconstruction(data, label, samples=2, multiply=1, dropout=0.3, sub_samples=1)
print(aug_data, aug_label)

[[[0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452
   0.05808361 0.86617615]
  [0.60111501 0.70807258 0.02058449 0.96990985 0.83244264 0.21233911
   0.18182497 0.18340451]
  [0.30424224 0.52475643 0.43194502 0.29122914 0.61185289 0.13949386
   0.29214465 0.36636184]]

 [[0.45606998 0.78517596 0.19967378 0.51423444 0.59241457 0.04645041
   0.60754485 0.17052412]
  [0.06505159 0.94888554 0.96563203 0.80839735 0.30461377 0.09767211
   0.68423303 0.44015249]
  [0.12203823 0.49517691 0.03438852 0.9093204  0.25877998 0.66252228
   0.31171108 0.52006802]]

 [[0.54671028 0.18485446 0.96958463 0.77513282 0.93949894 0.89482735
   0.59789998 0.92187424]
  [0.0884925  0.19598286 0.04522729 0.32533033 0.38867729 0.27134903
   0.82873751 0.35675333]
  [0.28093451 0.54269608 0.14092422 0.80219698 0.07455064 0.98688694
   0.77224477 0.19871568]]

 [[0.00552212 0.81546143 0.70685734 0.72900717 0.77127035 0.07404465
   0.35846573 0.11586906]
  [0.86310343 0.62329813 0.33089802 0.063558

In [1]:
import torch
import numpy as np
from dpeeg.data.functions import to_tensor
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset

device = torch.device('cuda:1')
batch_size = 128

def _data_loader(*datasets, keep_data_gpu : bool = True) -> DataLoader:
    '''Wrap multiple sets of data and labels and return DataLoader.

    Parameters
    ----------
    datasets : sequence of data (N, ...) and labels (N,)
        Sequence consisting of each piece of data and label. Can be ndarray
        and tensor.
    '''
    if len(datasets) == 0:
        raise ValueError('At least one dataset required as input.')

    # dataset wrapping tensors
    td = []
    for i, dataset in enumerate(datasets):
        if not (isinstance(dataset, (list, tuple)) and len(dataset) == 2):
            raise ValueError(f'Missing data or label in {i}th group.')

        data, label = to_tensor(dataset[0], dataset[1])
        if keep_data_gpu:
            data, label = data.to(device), label.to(device)
        td.append(TensorDataset(data, label))
    td = ConcatDataset(td)

    if keep_data_gpu:
        data_loader = DataLoader(td, batch_size, True)
    else:
        data_loader = DataLoader(td, batch_size, True)
    return data_loader

data = {
    1: {'train': [np.random.rand(588, 22, 1000), np.random.randint(2, size=588)],
        'test':  [np.random.rand(244, 22, 1000), np.random.randint(2, size=244)]},
    2: {'train': [np.random.rand(588, 22, 1000), np.random.randint(2, size=588)],
        'test':  [np.random.rand(244, 22, 1000), np.random.randint(2, size=244)]},
    3: {'train': [np.random.rand(588, 22, 1000), np.random.randint(2, size=588)],
        'test':  [np.random.rand(244, 22, 1000), np.random.randint(2, size=244)]},
}

train_loader = _data_loader(data[1]['train'], keep_data_gpu=True)
for data, label in train_loader:
    data, label = data.to(device), label.to(device)
    print(data.device, label.device)

/home/tao/miniconda3/envs/bci/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/tao/miniconda3/envs/bci/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


cuda:1 cuda:1
cuda:1 cuda:1
cuda:1 cuda:1
cuda:1 cuda:1
cuda:1 cuda:1
